In [1]:
# import ragas.metrics as m
import os
from dotenv import load_dotenv
import pandas as pd
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper

load_dotenv()
link = os.getenv('dsa_2214')
token = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = token

chapters = [
    'Data Structures and Algorithms',
    'Mathematical Preliminaries',
    'Algorithm Analysis',
    'Lists, Stacks, and Queues',
    'Binary Trees',
    'Non-Binary Trees',
    'Internal Sorting',
    'File Processing and External Sorting',
    'Searching',
    'Indexing',
    'Graphs',
    'Lists and Arrays Revisited',
    'Advanced Tree Structures',
    'Analysis Techniques',
    'Lower Bounds',
    'Patterns of Algorithms',
    'Limits to Computation',
]

In [2]:
data = pd.read_csv('data/sorting.csv')
data.columns = ['concept', 'outcome']
data

,concept,outcome
0,Data Structures->Basics->Algorithm Analysis,Apply time complexity analysis guideline to an...
1,Data Structures->Basics->Algorithm Analysis->O...,Demonstrate an understanding of big O notation...
2,Algorithms->Non-recursive Algorthims,Demonstrate an understanding of non-recursive ...
3,Algorithms->Non-recursive Algorithms->Search,Apply the Comparable interface for object comp...
4,Algorithms->Non-recursive Algorithms->Search->...,Demonstrate an understanding of linear search;...
5,Algorithms->Non-recursive Algorithms->Search->...,Demonstrate an understanding of binary search;...
6,Algorithms->Non-recursive Algorithms->Sort,Demonstrate an understanding of sorting;
7,Algorithms->Non-recursive Algorithms->Sort->In...,Demonstrate an understanding of insertion sort...
8,Algorithms->Non-recursive Algorithms->Sort->Se...,Demonstrate an understanding of selection sort...
9,Algorithms->Non-recursive Algorithms->Sort->Bu...,Demonstrate an understanding of bubble sort;An...


In [3]:
concept_data = data['concept'].tolist()
actual_concepts = []
for string in concept_data:
    words = string.split('->')
    for word in words:
        if word not in actual_concepts:
            actual_concepts.append(word)


outcome_data = data['outcome'].tolist()
actual_outcomes = []
for string in outcome_data:
    words = string.split(';')
    for s in words:
        if s not in actual_outcomes:
            actual_outcomes.append(s)

actual_concepts = [' '.join(actual_concepts)] * 4
actual_outcomes = [' '.join(actual_outcomes)] * 4

In [2]:
from src.extractor import relationExtractor
extractor = relationExtractor(link, token, chapters[6:10], os.getenv('connection_string'), 3000, 100, 'DocumentEmbeddings', '2214_embeddings')

<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [4]:
extractor.identify_concepts(1)[1]

{'Internal Sorting': ['comparisons. of course, radix sort in the end does not provide a more efﬁcient sorting algorithm than comparison - based sorting. thus, empirical evidence suggests that comparison - based sorting is a good approach. 3 the proof that any comparison sort requires ω ( n log n ) comparisons in the worst case is structured as follows. first, comparison - based decisions can be mod - eled as the branches in a tree. this means that any sorting algorithm based on com - parisons between records can be viewed as a binary tree whose nodes correspond to the comparisons, and whose branches correspond to the possible outcomes. next, the',
  'on the value of speciﬁc digits in the key value, so it is possible to take approaches to sorting that do not involve key comparisons. of course, radix sort in the end does not provide a more efﬁcient sorting algorithm than comparison - based sorting. thus, empirical evidence suggests that comparison - based sorting is a good approach. 3 th

In [5]:
concepts = extractor.identify_concepts(5)[0]
outcomes = extractor.identify_outcomes(5)[0]
kt = extractor.identify_key_terms(5)[0] 
dependencies = extractor.identify_dependencies(concepts)

In [6]:
kg = extractor.build_knowledge_graph(dependencies, concepts, outcomes, kt)
extractor.draw_knowledge_graph(kg)

./visualizations/kg.html


In [3]:
from langchain_core.documents import Document
from sentence_transformers import CrossEncoder

def rank_docs(queries_and_docs: list[tuple[str, str]], top_n: int) -> list[Document]:
    '''
    Ranks queries and documents using CrossEncoder

    Args:
        queries_and_docs (list[tuple[str, str]]): queries and documents
        top_n (int): number of docs to return

    Returns:
        list: list of top_n documents
    '''
    model = CrossEncoder(model_name = 'cross-encoder/ms-marco-MiniLM-L-6-v2')
    scores = model.predict([(t[0], t[1].page_content) for t in queries_and_docs])

    return sorted(list(zip(queries_and_docs, scores)), key = lambda x: x[1], reverse = True)[:top_n]

In [7]:
l = rank_docs([('give me a greeting!', Document('how are you today'))], 1)

In [9]:
l[0]

(('give me a greeting!',
  Document(metadata={}, page_content='how are you today')),
 -9.123926)

In [13]:
l[0][0][1]

Document(metadata={}, page_content='how are you today')